In [1]:
%load_ext autoreload
%autoreload 2

import json
import random
import torch
import numpy as np
from PIL import Image

import sys
sys.path.append("../detr")
import util.misc as utils
from eval import visualize

from main import get_transform, get_model

In [11]:
cmd_args = {
    "data_type": "structure",
    "config_file": "./structure_config.json",
    "model_load_path": "../models/pubtables1m_structure_detr_r18.pth",
    "debug_save_dir": "../data/PubTables-1M-Samples/preds",
    "table_words_dir": "../data/PubTables-1M-Samples",
}
config_args = json.load(open(cmd_args["config_file"], "rb"))
config_args["device"] = "cpu"

for key, value in cmd_args.items():
    if not key in config_args or not value is None:
        config_args[key] = value
args = type("Args", (object,), config_args)
# print(args.__dict__)

In [12]:
seed = args.seed + utils.get_rank()
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

device = args.device
model, criterion, postprocessors = get_model(args, device)

loading model from checkpoint


In [24]:
model.eval();

In [25]:
criterion.eval();

In [26]:
postprocessors

{'bbox': PostProcess()}

In [30]:
paths = [
    "../data/PubTables-1M-Samples/PMC493271_table_0.jpg",
    "../data/PubTables-1M-Samples/PMC493271_table_1.jpg",
    "../data/PubTables-1M-Samples/PMC493271_table_2.jpg",
    "../data/PubTables-1M-Samples/PMC493273_table_0.jpg",
    "../data/PubTables-1M-Samples/PMC493273_table_1.jpg",
    "../data/PubTables-1M-Samples/PMC493273_table_2.jpg",
]

for path in paths:
    target = {
        "img_path": path,
        "boxes":[],
    }

    img_path = target["img_path"]
    img = Image.open(img_path).convert("RGB")
    print("Image size:", img.size)

    transforms =  get_transform(args.data_type, "val")
    img_tensor, target = transforms(img, target)
    print("Tensor size:", img_tensor.size())

    outputs = model(img_tensor.unsqueeze(0))
    #print(outputs.keys())

    pred_logits = outputs["pred_logits"][0]
    pred_boxes = outputs["pred_boxes"][0]
    visualize(args, target, pred_logits, pred_boxes)

    #print(pred_logits.size())
    #print(pred_logits.softmax(-1).max(-1).indices)


Image size: (697, 346)
Tensor size: torch.Size([3, 496, 1000])
Image size: (629, 245)
Tensor size: torch.Size([3, 390, 1000])
Image size: (663, 211)
Tensor size: torch.Size([3, 318, 1000])
Image size: (695, 282)
Tensor size: torch.Size([3, 406, 1000])
Image size: (685, 372)
Tensor size: torch.Size([3, 543, 1000])
Image size: (628, 196)
Tensor size: torch.Size([3, 312, 1000])


In [ ]:
#import json
#import xmltodict
#
#with open("../data/PubTables-1M-Samples/PMC493271_table_0.xml") as xml_file:     
#    data_dict = xmltodict.parse(xml_file.read())     
#    json_data = json.dumps(data_dict)
#     
#    with open("../data/PubTables-1M-Samples/PMC493271_table_0.json", "w") as json_file:
#        json_file.write(json_data)